In [2]:
import pandas as pd
import numpy as np
import dynamic_portfolio.preprocess as prep
import dynamic_portfolio.utils as utils
import dynamic_portfolio.cross_validate as cv
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

warnings.filterwarnings(action='ignore')

In [23]:
y_pred = pd.read_csv('../raw_data/results/gradientboosting_pred.csv', index_col='date')
y_pred.drop(columns='GPN', inplace=True)

In [17]:
column_to_backtest = list(y_pred.columns)

In [12]:
tickers = utils.return_tickers() # loading all tickers
open_df = pd.DataFrame(index=y_pred.index)
counter=0
for ticker in tickers:
    counter+=1
    initial_df = utils.load_csv(f'{ticker}')
    try :
        initial_df.set_index(initial_df.columns[0], inplace=True)
        initial_df = initial_df[['open']]
        open_df[ticker] = initial_df["open"]
    except:
        pass
    print(f'done for {ticker} {counter}')
open_df = open_df.loc['2021-01-04':]

done for AAPL 1
done for MSFT 2
done for GOOG 3
done for AMZN 4
done for TSLA 5
done for UNH 6
done for XOM 7
done for JNJ 8
done for WMT 9
done for NVDA 10
done for JPM 11
done for V 12
done for CVX 13
done for PG 14
done for LLY 15
done for MA 16
done for HD 17
done for META 18
done for BAC 19
done for ABBV 20
done for PFE 21
done for KO 22
done for MRK 23
done for PEP 24
done for COST 25
done for ORCL 26
done for AVGO 27
done for TMO 28
done for MCD 29
done for CSCO 30
done for ACN 31
done for DHR 32
done for TMUS 33
done for ABT 34
done for WFC 35
done for DIS 36
done for LIN 37
done for NEE 38
done for BMY 39
done for NKE 40
done for VZ 41
done for TXN 42
done for UPS 43
done for COP 44
done for ADBE 45
done for CMCSA 46
done for CRM 47
done for PM 48
done for MS 49
done for AMGN 50
done for SCHW 51
done for HON 52
done for RTX 53
done for QCOM 54
done for T 55
done for IBM 56
done for DE 57
done for CVS 58
done for LOW 59
done for GS 60
done for UNP 61
done for NFLX 62
done for L

done for DXC 486
done for GNRC 487
done for AIZ 488
done for XRAY 489
done for LNC 490
done for DVA 491
done for MHK 492
done for LUMN 493
done for ALK 494
done for NWL 495
done for VNO 496
done for TAP 497


KeyError: "['GPN'] not found in axis"

In [19]:
column_to_backtest.remove('GPN')

In [25]:
open_df = open_df[column_to_backtest]

In [26]:
open_df.to_csv('open_price_454_stocks.csv')

In [ ]:
raw_data/backtest_data